In [1]:
from modules.binanceklines.binance_klines import *

from modules.postgrescrud.postgres_crud import PostgresCrud as PG
from datetime import datetime

In [2]:
#PG handler

DB_HOST='localhost'
POSTGRES_USER='pandazord'
POSTGRES_DB='pandazord_database'
POSTGRES_PASSWORD='06Fj@%r7KTXm5+eWn2'

pg = PG(DB_HOST, POSTGRES_DB, POSTGRES_USER, POSTGRES_PASSWORD)

In [ ]:
#Binance handler

asset_symbol = 'BTCUSDT'
candle_interval = '1m'
start_time = '1241893500000'
max_attempts = 10

binance_klines = GetKlines(asset_symbol, candle_interval, start_time, max_attempts)

In [20]:
def return_last_open_time(table_name):

    keys_dict = {'open_time': 'timestamp', 'open': 'numeric', 'high': 'numeric', 'low': 'numeric', 
                 'close': 'numeric', 'volume': 'numeric'}
    
    table_was_created = pg.create_table(table_name, keys_dict, pk='open_time')
    
    if (table_was_created): last_open_time_timestamp_format = 1241893500000
    
    else:
        
        try:
            
            last_open_time_datetime_format = pg.read_entries_from_table(table_name, 
                                                                        field_key = 'open_time', 
                                                                        sort_type = 'DESC', 
                                                                        limit = 1)[0][0]
            
            last_open_time_timestamp_format = int(1000*(datetime.timestamp(last_open_time_datetime_format)))
            
        except:
            
            last_open_time_timestamp_format = 1241893500000
    
    return last_open_time_timestamp_format

In [22]:
#Test return_last_open_time

asset_symbol = 'ETHUSDT'
#asset_symbol = 'EOSUSDT'

table_name = 'binance_klines_' + asset_symbol + '_' + '1m'


last_open_time = return_last_open_time(table_name)

last_open_time

1502953320000

# Atribuições

## main.py

Ciclar lendo a tabela de pares

- A) Para os pares cujo "automatically update" seja "yes":  
    
    
> 1 - "status = "building", verificar se o PID está ativo. Caso não esteja ativo, chamar o "continuous_update_of_historical_data.py" em segundo plano (atualizando a tabela de pares, no respectivo par, com o PID do processo chamado). Caso esteja ativo, bypassar;  
    
 > 2 - "status" = "absent" ou "full", chamar o "continuous_update_of_historical_data.py" em segundo plano (atualizando a tabela de pares, no respectivo par, com o PID do processo chamado).    

- Caso o "automatically update" seja seja "no", bypassar.  

## continuous_update_of_historical_data.py

- A) Buscar a tabela de dados do ativo em questão

> 1 - Tabela ausente: 
>> 1.1 - Cria a tabela;  
>> 1.2 - Puxa a primeira leva de dados junto à binance, com um start_time muito antigo;  
>> 1.3 - Procede a verifiação e eventual correção de dados faltantes;  
>> 1.4 - Salva na respectiva tabela do banco;  
>> 1.5 - Segue para 2;  

> 2 - Tabela presente:
>> 2.1 - A partir do timestamp do último candle salvo, faz a solicitação da próxima leva de dados  
>> 2.2 - Procede a verifiação e eventual correção de dados faltantes;  
>> 2.3 - Salva na respectiva tabela do banco;  
>> 2.4 - Marca os status como "full"

In [25]:
#TODO: Docstrings

import os
import sys
import time
from datetime import datetime
import requests
from modules.postgrescrud.postgres_crud import PostgresCrud as PG
from modules.binanceklines.binance_klines import *

#pair = str(sys.argv[1])
asset_symbol = 'BTCUSDT'

#interval = str(sys.argv[2])
candle_interval = '1m'

#start_time = str(sys.argv[3])
#start_time = '1241893500000'

table_name = 'binance_klines_' + asset_symbol + '_' + candle_interval

keys = {'open_time': 'timestamp', 'open': 'numeric', 'high': 'numeric', 'low': 'numeric', 'close': 'numeric',
    'volume': 'numeric'}

max_attempts = 10 #TODO: Pode vir de um parâmetro depois

was_entry_updated_to_building = pg.update_entry('binance_assets', 
                                                'asset_symbol', asset_symbol, 'status', 'building')

if not (was_entry_updated_to_building): pass #TODO: Tratar exceção

round_counter = 1

last_open_time = return_last_open_time(table_name)

while True:

    klines = []
    
    start_time = str(last_open_time + 30000)  #Um passo de 30s (30000 milissegundos
        
    klines = GetKlines(asset_symbol, candle_interval, start_time, max_attempts).formated()
    
    if (len(klines) > 0):
        
        if (len(klines) < 500): #Chegou na kline mais recente
            
            klines = klines[:(len(klines)-1)] #Apaga o último resgistro (candle não fechado)
            
            if (len(klines) == 0): break
        
        #PAREI AQUI
        treated_klines, last_open_time = treat_for_missing_data(last_open_time, klines)
        
        
        ###
               
        save_in_table_job_status = pg.save_data_in_table(table_name, keys, treated_klines)
        
        data_completer_job_status = 'done'
        
        break

        if (len(klines) > 500): break #TODO: Isto é uma anomalia, deve ser tratada
            
    else: #TODO: Tratar exceção (Falha de comunicação com a binance, klines nulas)
        
        pass
    
    response_ping = requests.get('https://api.binance.com/api/v1/ping')

    limite_of_requests = bool (int(response_ping.headers['X-MBX-USED-WEIGHT']) >= 1100)
    
    if(limite_of_requests): time.sleep(60)

    round_counter+=1

was_entry_updated_to_full = pg.update_entry('binance_assets', 'asset_symbol', asset_symbol, 'status', 'full')

if not (was_entry_updated_to_full): pass #TODO: Tratar exceção

NameError: name 'treat_for_missing_data' is not defined

In [ ]:
response_ping = requests.get('https://api.binance.com/api/v1/ping')

limite_of_requests = bool (int(response_ping.headers['X-MBX-USED-WEIGHT']) >= 1100)

limite_of_requests

In [ ]:
#DRAFT: criando lista auxiliar de open_times

open_time_list_in = []

for i in range(len(klines) - 1):
    
    open_time_entry = klines[i][0]
    
    #print (open_time_entry)
    
    open_time_list_in.append(open_time_entry)

In [ ]:
def aux_open_time(data_in):
    
    aux_open_time_list = []
    
    current_open_time = data_in.loc[0, 'open_time']

    last_open_time = data_in.loc[len(data_in)-1, 'open_time']
    
    while (current_open_time < last_open_time):
        
        aux_open_time_list.append(current_open_time)
        
        current_open_time = datetime.fromtimestamp(datetime.timestamp(current_open_time) + 60)
        
    return aux_open_time_list

In [ ]:
# Tabelas testes

# 1 - create_table test

asset_symbol = 'EOSUSDT'

test_table_name = 'binance_klines_' + asset_symbol + '_' + candle_interval

keys_dict = {'open_time': 'timestamp', 'open': 'numeric', 'high': 'numeric', 'low': 'numeric', 
             'close': 'numeric', 'volume': 'numeric'}

table_was_created = pg.create_table(test_table_name, keys_dict ,pk='open_time')

if table_was_created: print('Tabela {} criada com sucesso'.format(test_table_name))

In [ ]:
# 2 - save_data_in_table test

import datetime

test_data = [[datetime.datetime(2017, 8, 17, 4, 0), '4261.48', '4261.48', '4261.48',
              '4261.48', '1.77518300'],
                [datetime.datetime(2017, 8, 17, 4, 1), '4261.48', '4261.48', '4261.48',
              '4261.48', '1.77518300'],
                [datetime.datetime(2017, 8, 17, 4, 2), '4261.48', '4261.48', '4261.48',
              '4261.48', '1.77518300']]

data_was_saved_in_table = pg.save_data_in_table(test_table_name, keys_dict, test_data)

if data_was_saved_in_table: print('Dados gravados na tabela {} com sucesso'.format(test_table_name))

In [ ]:
#Para anexar aos testes

from copy import deepcopy

def baguncar_klines(klines_in):
    
    klines_out = deepcopy(klines_in)
    
    for i in range(len(klines_in)):
      
        klines_out[i][0] = (klines_in[i][0] + 40000)
    
    return klines_out

In [ ]:
#Trash

